In [48]:
# Imports
import polars as pl 
import os


print(os.getcwd())


/mnt/c/ECON412/class_project/ECON_412_Project_alw_fork


If the current working directory is different from what you expected, you can change it using the os module

In [49]:
os.chdir("/mnt/c/ECON412/class_project/ECON_412_Project_alw_fork") # aboslute file path
print(os.getcwd())

/mnt/c/ECON412/class_project/ECON_412_Project_alw_fork


In [50]:
# Sample file to explore
filename = "Data/Ticker_Data/MMM.csv"

#skips the metadata at the top of the CSV
df_test = pl.read_csv(filename, skip_rows=4, has_header=True)

#renames the header
df_test.columns = ["Date", "Close", "High", "Low", "Open", "Volume"]

df_test.head()

Date,Close,High,Low,Open,Volume
str,f64,f64,f64,f64,i64
"""2015-01-06""",92.939262,94.532673,92.406176,94.210485,4224272
"""2015-01-07""",93.612999,93.894189,93.109195,93.671574,3685235
"""2015-01-08""",95.856674,95.891826,94.034799,94.11095,3758908
"""2015-01-09""",94.679169,96.073411,94.47414,95.985532,2844806
"""2015-01-12""",94.163651,95.130239,93.741866,95.130239,2570443


In [51]:
# Number of rows and columns
df_test.shape

(2514, 6)

In [52]:
# Observes schema
df_test.schema

Schema([('Date', String),
        ('Close', Float64),
        ('High', Float64),
        ('Low', Float64),
        ('Open', Float64),
        ('Volume', Int64)])

In [53]:
# Number of nul values per column
df_test.null_count()

Date,Close,High,Low,Open,Volume
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


TODO: change 'Date' from 'String' to 'DateTime'

In [ ]:
# TODO: here
df_test = df_test.with_columns(pl.col("Date").str.strptime("%Y-%m-%d"))
df_test.schema

NameError: name 'df_test' is not defined

TODO: Add 'Ticker', 'Sector', and 'Sub-Industry' columns from company_list.csv

In [55]:
# TODO: here

TODO: Verify that all CSVs have the same characteristics (shape, column names, null count), also ochange their 'Date' dtype

In [56]:
# TODO: here

TODO: load in company info

In [ ]:
# TODO: here

TODO: combine company info and ticker df

In [ ]:
# TODO: here

TODO: create characteristics(df) with 'assert'

In [ ]:
# TODO: here